# Module 1, Section 3: Multi-Agent Architecture

<div align="center">
    <img src="../../images/supervisor_agent.png" width="800">
</div>

In this section, we'll build a multi-agent customer support system using:
- **Specialized sub-agents** focused on distinct domains (database vs. documents)
- **Supervisor agent** that routes and re-writes queries for the correct expert
- **Tool wrapping** enabling the supervisor to delegate tasks to sub-agents as tools
- **Testing with LangSmith traces** to see multi-agent coordination in action

By the end, we'll have a working system with:
- **Database Agent** for order, product, and customer info queries
- **Documents Agent** for searching product documents and policies
- **Supervisor** for orchestration and delegation

## Setup

Load environment variables:

In [ ]:
import uuid
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

## 1. Import Tools

We'll use the tools we created in previous sections, plus new tools for document search. 

In particular, we've added two tools — `search_product_docs` and `search_policy_docs` — that let our agent search over product information and company policies using semantic search (vector database + embeddings).

<div align="center">
    <img src="../../images/db_rag_tools.png">
</div>

In [ ]:
from tools.database import (
    get_order_status,
    get_order_items,
    get_product_info,
    get_order_item_price,
)
from tools.documents import search_product_docs, search_policy_docs

## 2. Build Documents Agent

Our first specialist: an agent focused on searching product documentation and policies.

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from config import DEFAULT_MODEL

# Initialize model
llm = init_chat_model(DEFAULT_MODEL)

# Create Documents Agent
docs_agent = create_agent(
    model=llm,
    tools=[search_product_docs, search_policy_docs],
    name="docs_agent",
    system_prompt="""You are the company policy and product information specialist for TechHub customer support.

Your role is to answer queries from a supervisor agent about product specifications, features, compatibility, 
policies (returns, warranties, shipping), and setup instructions given the tools you have been provided.
You do NOT interact directly with customers, you only interact with the supervisor agent.

Capabilities: Search product documentation and company policies.

Instructions:
- Always search the documentation to provide accurate, detailed information.
- If information is missing or not found, say so clearly.
- Do NOT make assumptions or provide information not explicitly present in the documentation.

Be accurate, concise, and specific in your replies.""",
    checkpointer=MemorySaver(),
)

## 3. Build Database Agent

Our second specialist: an agent focused on querying structured data from the TechHub database (order status, order items, product info).

In [ ]:
# Create Database Agent
db_agent = create_agent(
    model=llm,
    tools=[get_order_status, get_order_items, get_product_info, get_order_item_price],
    name="db_agent",
    system_prompt="""You are the database specialist for TechHub customer support.

Your role is to answer queries from a supervisor agent about orders or products using the TechHub database tools you have been provided.
You do NOT interact directly with customers, you only interact with the supervisor agent.

Capabilities: Look up and report order status, order details (items, quantities), product prices, and product availability.

Instructions:
- Always retrieve answers directly from the database using the available tools.
- If information is missing or not found, say so clearly.
- Do NOT make assumptions or provide information not explicitly present in the database.

Be accurate, concise, and specific in your replies.""",
    checkpointer=MemorySaver(),
)

## 4. Build Supervisor Agent

Now we'll create a supervisor agent that:
- interacts with the end user
- reasons about their request
- formulates queries for the sub-agents
- synthesizes responses from the sub-agents
- responds appropriately

**Key insight:** Sub-agents become *tools* for the supervisor!

In [ ]:
from langchain_core.tools import tool


# Wrap Database Agent as a tool
@tool(
    "database_specialist",
    description="Query TechHub database specialist for order status, order details, product prices, and product availability",
)
def call_database_specialist(query: str) -> str:
    """Call the database specialist subagent.

    Args:
        query: The question to ask the database specialist
    """
    result = db_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content


# Wrap Documents Agent as a tool
@tool(
    "documentation_specialist",
    description="Query TechHub documentation specialist to search for product specs, policies, warranties, and setup instructions",
)
def call_documentation_specialist(query: str) -> str:
    """Call the documentation specialist subagent.

    Args:
        query: The question to ask the documentation specialist
    """
    result = docs_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content

The tool descriptions help the supervisor decide when to use each tool, so make them clear and specific. We return only the sub-agent’s final response, as the supervisor doesn’t need to see intermediate reasoning or tool calls. This helps maintain a clean context window for the supervisor.

Now, lets create the supervisor agent!

In [ ]:
# Create Supervisor Agent
supervisor_agent = create_agent(
    model=llm,
    tools=[call_database_specialist, call_documentation_specialist],
    name="supervisor_agent",
    system_prompt="""You are a supervisor agent for TechHub customer support.

Your role is to interact with customers to understand their questions, use the sub-agent tools provided to 
gather information needed to answer their questions, and then provide helpful responses to the customer.

Capabilities:
- Interact with customers to understand their questions
- Use database_specialist to help answer questions about orders (status, details) and products (prices, availability)
- Use documentation_specialist to help answer questions about product specs, policies, warranties, and setup instructions

You can use multiple tools if needed to fully answer the question.
Always provide helpful, accurate, concise, and specific responses to customer questions.""",
    checkpointer=MemorySaver(),
)

## 5. Test the Supervisor Agent

We can test inline below or with [LangSmith Studio](https://docs.langchain.com/oss/python/langgraph/studio#langsmith-studio) for a nicer experience.

**To visualize with LangSmith Studio:**
1. Run `langgraph dev` in your terminal from the workshop root directory
2. Open the provided local URL (typically `http://localhost:8000`)
3. Select your agent graph to see an interactive visualization

**What LangSmith Studio provides:**
- Visual graph representation of your agent's workflow
- Real-time execution tracking as messages flow through nodes
- Interactive testing interface with streaming responses
- Step-by-step debugging of agent decisions and tool calls

### Test Simple Routing

Let's test the supervisor with queries that need just ONE specialist:

In [ ]:
print("Query 1: Order status (should route to Database Agent)")

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2025-0030?"}
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print(
    "\n💡 Check LangSmith traces to see: supervisor → database_specialist → supervisor"
)

In [ ]:
print("\nQuery 2: Product question (should route to Documents Agent)")

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's included in the box with the Logitech MX Keys keyboard?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print(
    "\n💡 Check LangSmith traces to see: supervisor → documentation_specialist → supervisor"
)

### Test Multi-Agent Coordination

Now the more interesting part - queries that require BOTH sub-agents!

**Parallel Execution Example:**

In [ ]:
print("Query 3: Requires both Database AND Documents agents, can be parallelized")

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Is the MacBook Air in stock? What type of processor does it have? And if I buy it, what's the return policy?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print("\n💡 Check LangSmith traces to see the parallel flow")

**Sequential Execution Example:** 

Query that requires SEQUENTIAL agent execution - output from first agent feeds into second agent. This demonstrates true agent orchestration where the supervisor can't parallelize!

In [ ]:
print("Query 4: Requires SEQUENTIAL coordination (DB → Documents)")

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I bought a monitor in my last order (ORD-2024-0063). What kind was it? Is the MacBook Air compatible with it?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print("\n💡 Check LangSmith traces to see SEQUENTIAL flow")

#### 📦 Code Refactoring Note

The agents we built in this section (Database Agent, Documents Agent, and Supervisor) have been **refactored into the `agents/` directory** as reusable factory functions:

- `agents/db_agent.py` - Database Agent factory
- `agents/docs_agent.py` - Documents Agent factory
- `agents/supervisor_agent.py` - Supervisor Agent factory

**Why factory functions?**
- Fresh checkpointer for each instantiation (no state pollution)
- Clean imports and reusability across notebooks

In **Section 4**, we'll import these agents rather than redefining them:
```python
from agents import create_db_agent, create_docs_agent, create_supervisor_agent
```